In [1]:
import os

In [2]:
%pwd

'd:\\ineuron\\Project_2 - Text summarizer\\Text_Summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\ineuron\\Project_2 - Text summarizer\\Text_Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path #reading root directory
    data_path: Path
    tokenizer_name: Path

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])  #this is used to create data_transformation directory inside the artifacts directory

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [8]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

c:\Users\Apoorv.DESKTOP-3VQL5IT\miniconda3\envs\textS\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-07-03 13:06:40,240: INFO: config: PyTorch version 2.3.1 available.]


In [9]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config #assigning configurations here
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name) #tokenizer name from where we want to tokenize
        #autotokenizer loads pre-trained tokenizer, It does:
        #Splitting text into individual words or sub-word units (tokens).
        #Mapping these tokens to unique integer IDs the model can understand.
        #Adding special tokens (e.g., padding tokens) for specific tasks.
    

    
    def convert_examples_to_features(self,example_batch):#taking batch of examples from input
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
        #taking 'dialogue' text from example batch, max length of encoded text is 1024
        # truncation=True indicates that if the dialogue exceeds 1024 tokens, it will be truncated (cut off) to fit the limit.
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],# The list of integer IDs representing the encoded dialogue tokens.
            'attention_mask': input_encodings['attention_mask'], # binary mask indicating which parts of the dialogue are relevant (1) and which can be ignored (0) by the model (useful for padding)
            'labels': target_encodings['input_ids'] #The list of integer IDs representing the encoded summary tokens (treated as labels for training)
        }
    

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path) #here we are loading the samsum dataset from disk
        #samsum dataset has 3 columns: "id", "dialogue", "summary" in apache arrow format
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)#map the above function to each 
        #batch of the dataset as here we have kept batched = True
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset")) #saving the data after transformation on the disk

In [ ]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e